Find the markdown blocks that say interaction required! The notebook should take care of the rest!

# Import libs

In [1]:
import sys
import os
sys.path.append('..')
from eflow.foundation import DataPipeline,DataFrameTypes
from eflow.data_analysis import FeatureAnalysis, NullAnalysis
from eflow.model_analysis import ClassificationAnalysis
from eflow.data_pipeline_segments import FeatureTransformer, DataEncoder, FeatureDataCleaner
from eflow.utils.modeling_utils import optimize_model_grid
from eflow.utils.eflow_utils import get_type_holder_from_pipeline, remove_unconnected_pipeline_segments
from eflow.utils.math_utils import get_unbalanced_threshold
from eflow.utils.sys_utils import create_dir_structure
from eflow.utils.eflow_utils import create_color_dict_for_features
from eflow.utils.pandas_utils import missing_values_table,data_types_table, value_counts_table, suggest_removal_features 
from eflow.widgets import ColorLabelingWidget

import pandas as pd
import numpy as np
import scikitplot as skplt
import matplotlib.pyplot as plt
import copy
from IPython.display import clear_output
from IPython.core.getipython import get_ipython


In [2]:
# # Additional add ons
# !pip install pandasgui
# !pip install pivottablejs
# clear_output()

In [3]:
%matplotlib notebook
%matplotlib inline

## Juypter notebook generating cells

### Important Note: Replace if set to True will remove all the contents of whatever cell it is called in. But it can be undone with a simple CMD + Z. 🙂

In [4]:
# Author: http://tinyurl.com/y6mghyzl
def create_new_cell(contents,
                    replace=False):
    """
    Desc:
        Creates a new jupyter cell.
    """
    shell = get_ipython()
    shell.set_next_input(contents,
                         replace=replace)

def __format_list_to_string(list_name,
                            list_contents):
    """
    Desc:
        Converts a list to a string and adds newlines for formating.
    """
    output_str = f"{list_name} = ["
    escape_seq_count = 0
    final_index = len(list_contents) - 1
    req_spacing = len(output_str)

    for i,element in enumerate(list_contents):
        if i == final_index:
            if isinstance(element,str):
                output_str += f'\"{element}\"'
            else:
                output_str += f'{element}'
        else:

            if isinstance(element,str):
                output_str += f'\"{element}\",'
            else:
                output_str += f'{element},'
        
        if len(output_str.split("\n")[escape_seq_count]) > 78:
            output_str += "\n"
            output_str += (" " * req_spacing)
            escape_seq_count += 1
    output_str += "]"
    return output_str

def create_new_cell_with_removal_features(df,
                                          replace=True):
    """
    Desc:
        Creates a new cell block with a list of suggested features to remove.
    
    Args:
        df:
            Pandas DataFrame object
            
        replace:
            Boolean to determine replacing the current cell.
    """
    
    # Get suggestions for removal
    cell_content = __format_list_to_string("removal_features",
                                           suggest_removal_features(df))
    # Add a sort of calling card of the function that created it
    cell_content = f"# create_new_cell_with_removal_features(df,replace={replace})\n" + cell_content
    create_new_cell(cell_content,
                    replace=replace)

def create_new_cell_with_null_removal_features(df,
                                               null_threshold=.25,
                                               replace=True):
    """
    Desc:
        Creates a new cell block with a list of suggested features to remove based on nulls.
    
    Args:
        df:
            Pandas DataFrame object
            
        null_threshold:
            Any features that contain x% percent of nulls are suggested.
            
        replace:
            Boolean to determine replacing the current cell.
    """
    mis_val = df.isnull().sum()
    mis_val_percent = df.isnull().sum() / len(df)
    
    cell_content = f"# create_new_cell_with_null_removal_features(df,null_threshold={null_threshold},replace={replace})\n"
    cell_content += __format_list_to_string("remove_null_features",
                                            mis_val_percent[mis_val_percent > null_threshold].index.to_list())
    # Add a calling card of the function that created it
    
    create_new_cell(cell_content,
                    replace=replace)

def create_new_cell_with_feature_value_color_dict(df,
                                                  df_features,
                                                  value_limit=50,
                                                  replace=True):
    """
    Desc:
        Creates a new cell block with a dict of suggested feature value colors.
    
    Args:
        df:
            Pandas DataFrame object
        
        df_features:
            DataFrameTypes object.
        
        null_threshold:
            Any features that contain x% percent of nulls are suggested.
            
        value_limit:
            Limit the amount of feature_values until the system will ignore
            the feature all together for dict generation.
            
        replace:
            Boolean to determine replacing the current cell.
    """
    feature_value_color_dict = create_color_dict_for_features(df,
                                                              df_features,
                                                              value_limit)
    # Add a sort of calling card of the function that created it
    cell_content = ""
    cell_content += f"# create_new_cell_with_feature_value_color_dict(df,df_features,value_limit={value_limit},replace={replace})\n"
    cell_content += "feature_value_color_dict=dict()"
    feature_count = 0
    for feature_name, feature_value_color in feature_value_color_dict.items():
        if feature_value_color_dict[feature_name].keys(): 
            cell_content += f"\nfeature_value_color_dict[\"{feature_name}\"] = dict()"
        else:
            cell_content += f"\n\n# The feature '{feature_name}' has to many values! Asserting assumption that you don't want to give colors to each!"
        
        for feature_value, color in feature_value_color.items():

            color = feature_value_color_dict[feature_name][feature_value]
            
            if feature_name in df_features.bool_features() or feature_name in df_features.categorical_features():
                try:
                    feature_value = int(float(feature_value))
                except:
                    pass
            
            if isinstance(feature_value,str):
                feature_value = f"\"{feature_value}\""
            else:
                feature_value = f"{feature_value}"
            
            if color is None:
                cell_content += f"\nfeature_value_color_dict[\"{feature_name}\"][{feature_value}] = None"
            else:
                cell_content += f"\nfeature_value_color_dict[\"{feature_name}\"][{feature_value}] = \"{color}\""
        cell_content += "\n"
        
    create_new_cell(cell_content,
                    replace=replace)

def create_new_cell_with_categorical_dict(df,
                                          df_features,
                                          value_limit=50,
                                          replace=True):
    """
    Desc:
        Creates a new cell block with a dict of
    
    Args:
        df:
            Pandas DataFrame object

        df_features:
            DataFrameTypes object.

        value_limit:
            Limit the amount of feature_values until the system will ignore
            the feature all together for dict generation.
            
        replace:
            Boolean to determine replacing the current cell.
    """

    cell_content = ""
    cell_content += f"# create_new_cell_with_categorical_dict(df,df_features,value_limit={value_limit},replace={replace})\n"
    cell_content += "categorical_value_dict = dict()\n"
    
    categorical_value_dict = dict()
    for feature_name in df_features.categorical_features():
        
        # Find and sort feature values
        feature_values = df[feature_name].value_counts(sort=False).index.to_list()
        feature_values = [str(val) for val in feature_values]
        feature_values.sort()
        
        # Create feature cat dict
        cat_found = False
        categorical_value_dict[feature_name] = dict()
        for val in feature_values:
            try:
                categorical_value_dict[feature_name][int(val)] = ""
                cat_found = True
            except ValueError:
                pass
        
        # Delete feature name if no categories are found
        if not cat_found:
            del categorical_value_dict[feature_name]
    
    for feature_name,cat_val_dict in categorical_value_dict.items():
        
        if len(cat_val_dict.keys()) < value_limit:
            cell_content += f"categorical_value_dict[\"{feature_name}\"]=dict()\n"
            for cat,val in cat_val_dict.items():

                if isinstance(val,str):
                    cell_content += f"categorical_value_dict[\"{feature_name}\"][{cat}] = \"{val}\"\n"
                else:
                    cell_content += f"categorical_value_dict[\"{feature_name}\"][{cat}] = {val}\n"
        else:
            cell_content += f"\n\n# The feature '{feature_name}' has to many values! Asserting assumption that you don't want to give encode to each!"

        

    create_new_cell(cell_content,
                    replace=replace)
    
    

def create_new_cell_with_value_representation(df,
                                              df_features,
                                              value_limit=50,
                                              replace=True):
    """
    Desc:
        Creates a new cell block with a dict of suggested feature value colors.
    
    Args:
        df:
            Pandas DataFrame object

        df_features:
            DataFrameTypes object.

        value_limit:
            Limit the amount of feature_values until the system will ignore
            the feature all together for dict generation.
            
        replace:
            Boolean to determine replacing the current cell.
    """
    feature_value_representation = dict()
    for feature_name in df_features.string_features():
        feature_value_representation[feature_name] = dict()
        for val in df[feature_name].dropna().value_counts(sort=False).index.to_list():
            if isinstance(val,str):
                if len(val) == 0:
                    continue
                if len(val) <= 3 or val not in words.words():
                    feature_value_representation[feature_name][val] = ""

                if len(feature_value_representation[feature_name].keys()) >= 50:
                    break

        if not len(feature_value_representation[feature_name].keys()):
            del feature_value_representation[feature_name]
    cell_content = ""
    cell_content += f"# create_new_cell_with_value_representation(df,df_features,value_limit={value_limit},replace={replace})\n"
    
    cell_content += "feature_value_representation = dict()\n"
    for feature_name,val_repr_dict in feature_value_representation.items():
        
        if len(val_repr_dict.keys()) < value_limit:
            cell_content += f"feature_value_representation[\"{feature_name}\"] = dict()\n"
            for val,reprs in val_repr_dict.items():

                if isinstance(val,str):
                    cell_content += f"feature_value_representation[\"{feature_name}\"][\"{val}\"] = "
                else:
                    cell_content += f"feature_value_representation[\"{feature_name}\"][{val}] = "
                
                if isinstance(reprs,str):
                    cell_content += f"\"{reprs}\"\n"
                else:
                    cell_content += f"{reprs}\n"
        else:
            cell_content += f"\n\n# The feature '{feature_name}' has to many values! Asserting assumption that you don't want to give representation to to each!"
        
        cell_content += "\n"
    create_new_cell(cell_content,
                    replace=replace)

def create_new_cell_with_binned_features(df,
                                         df_features,
                                         bins=5,
                                         replace=True):
    """
    Desc:
        Creates a new cell block with a list of suggested bins and labels for each feature.
    
    Args:
        df:pd.Dataframe
            Pandas DataFrame object.
        
        df_features:
            DataFrameTypes object.
            
        bins:int
            The amount of bins to give to apply to each feature
            
        replace:bool
            Boolean to determine replacing the current cell.
    """
    
    # Add a sort of calling card of the function that created it
    cell_content = f"# create_new_cell_with_binned_features(df,df_features,bins={bins},replace={replace})\n"
    
    for feature_name in df_features.continuous_numerical_features():
        bins,labels = auto_binning(df,
                                   df_features,
                                   feature_name,
                                   bins=5)
        print(bins)
        cell_content += f"feature_name = \"{feature_name}\"\n"
        cell_content += __format_list_to_string("bins",
                                                bins)
        cell_content += "\n"
        cell_content += __format_list_to_string("labels",
                                                labels)
        
        cell_content += f"\ndf_features.set_feature_binning(feature_name,\n"
        cell_content += "                                bins,\n"
        cell_content += "                                labels)\n"
        cell_content += "\n\n"
    
    create_new_cell(cell_content,
                    replace=replace)

## Declare Project Variables

### Interaction required

In [5]:
dataset_path = "Datasets/train.csv"

# -----
dataset_name = "Home Insurance Cross Sell Prediction"
pipeline_name = "testing stuff"
# -----

# -----
notebook_mode = True

## Clean out segment space

In [6]:
remove_unconnected_pipeline_segments()

# Import dataset

In [7]:
df = pd.read_csv(dataset_path)
shape_df = pd.DataFrame.from_dict({'Rows': [df.shape[0]],
                                   'Columns': [df.shape[1]]})
display(shape_df)
display(df.head(30))

Rows  Columns
0  381109       12

id  Gender  Age  Driving_License  Region_Code  Previously_Insured  \
0    1    Male   44                1         28.0                   0   
1    2    Male   76                1          3.0                   0   
2    3    Male   47                1         28.0                   0   
3    4    Male   21                1         11.0                   1   
4    5  Female   29                1         41.0                   1   
5    6  Female   24                1         33.0                   0   
6    7    Male   23                1         11.0                   0   
7    8  Female   56                1         28.0                   0   
8    9  Female   24                1          3.0                   1   
9   10  Female   32                1          6.0                   1   
10  11  Female   47                1         35.0                   0   
11  12  Female   24                1         50.0                   1   
12  13  Female   41                1         15.0                   1   
13  14    Male   76                1         28.0                   0   
14  15    Male   71                1         28.0                   1   
15  16    Male   37                1          6.0                   0   
16  17  Female   25                1         45.0                   0   
17  18  Female   25                1         35.0                   1   
18  19    Male   42                1         28.0                   0   
19  20  Female   60                1         33.0                   0   
20  21    Male   65                1         28.0                   0   
21  22    Male   49                1         28.0                   0   
22  23    Male   23                1         50.0                   1   
23  24    Male   44                1         28.0                   0   
24  25    Male   34                1         15.0                   1   
25  26  Female   21                1         28.0                   1   
26  27  Female   51                1         28.0                   0   
27  28    Male   26                1          6.0                   1   
28  29  Female   25                1          8.0                   1   
29  30    Male   57                1         11.0                   0   

   Vehicle_Age Vehicle_Damage  Annual_Premium  Policy_Sales_Channel  Vintage  \
0    > 2 Years            Yes         40454.0                  26.0      217   
1     1-2 Year             No         33536.0                  26.0      183   
2    > 2 Years            Yes         38294.0                  26.0       27   
3     < 1 Year             No         28619.0                 152.0      203   
4     < 1 Year             No         27496.0                 152.0       39   
5     < 1 Year            Yes          2630.0                 160.0      176   
6     < 1 Year            Yes         23367.0                 152.0      249   
7     1-2 Year            Yes         32031.0                  26.0       72   
8     < 1 Year             No         27619.0                 152.0       28   
9     < 1 Year             No         28771.0                 152.0       80   
10    1-2 Year            Yes         47576.0                 124.0       46   
11    < 1 Year             No         48699.0                 152.0      289   
12    1-2 Year             No         31409.0                  14.0      221   
13    1-2 Year            Yes         36770.0                  13.0       15   
14    1-2 Year             No         46818.0                  30.0       58   
15    1-2 Year            Yes          2630.0                 156.0      147   
16    < 1 Year            Yes         26218.0                 160.0      256   
17    < 1 Year             No         46622.0                 152.0      299   
18    1-2 Year            Yes         33667.0                 124.0      158   
19    1-2 Year            Yes         32363.0                 124.0      102   
20    1-2 Year            Yes         41184.0               

In [8]:
data_types_table(df)

Data Types
Features                       
Region_Code             float64
Annual_Premium          float64
Policy_Sales_Channel    float64
id                        int64
Age                       int64
Driving_License           int64
Previously_Insured        int64
Vintage                   int64
Response                  int64
Gender                   object
Vehicle_Age              object
Vehicle_Damage           object

# Loading and init df_features

In [9]:
# Option: 1
# df_features = get_type_holder_from_pipeline(pipeline_name)

In [10]:
# Option: 2
df_features = DataFrameTypes()
df_features.init_on_json_file(os.getcwd() + f"/eflow Data/{dataset_name}/df_features.json")

In [11]:
df_features.display_features(display_dataframes=True,
                             notebook_mode=notebook_mode)

Data Types
Features                     
Gender                 string
Vehicle_Age            string
Response                 bool
Previously_Insured       bool
Vehicle_Damage           bool
Annual_Premium        integer
Age                   integer
Vintage               integer

# Any extra processing before eflow DataPipeline

# Setup pipeline structure

In [12]:
main_pipe = DataPipeline(pipeline_name,
                         df,
                         df_features,
                         remove_past_contents=True)

Removing the feature: "Policy_Sales_Channel"
Removing the feature: "Driving_License"
Removing the feature: "id"
Removing the feature: "Region_Code"


In [13]:
throw_error = False
for feature in df.columns:
    if feature in df_features.integer_features():
        try:
            df[feature] = df[feature].astype("int")
        except:
            error_msg = f"An error has occured when converting {feature} to an integer series."
            if throw_error:
                raise ValueError(error_msg)
            else:
                print(error_msg)

    elif feature in df_features.float_features():
        try:
            df[feature] = df[feature].astype("float")
        except:
            error_msg = f"An error has occured when converting {feature} to an float series."
            if throw_error:
                raise ValueError(error_msg)
            else:
                print(error_msg)
                
    elif feature in df_features.categorical_features():
        try:
            df[feature] = df[feature].astype('category')
        except:
            error_msg = f"An error has occured when converting {feature} to an category series."
            if throw_error:
                raise ValueError(error_msg)
            else:
                print(error_msg)

    elif feature in df_features.datetime_features():
        try:
            df[feature] = pd.to_datetime(df[feature])
        except:
            error_msg = f"An error has occured when converting {feature} to an datetime."
            if throw_error:
                raise ValueError(error_msg)
            else:
                print(error_msg)
                
    elif feature in df_features.string_features():
        try:
            df[feature] = df[feature].astype("object")
        except:
            error_msg = f"An error has occured when converting {feature} to an string."
            if throw_error:
                raise ValueError(error_msg)
            else:
                print(error_msg)

## Remove Unwanted Columns due to illogical nulls

### Any features that have to many nulls/we can't or shouldn't perform any special logic to determine the closest or actual value

In [14]:
# create_new_cell_with_null_removal_features(df,null_threshold=0.25,replace=True)
remove_null_features = []

### Add to main pipeline

In [15]:
if len(remove_null_features):
    feature_transformer = FeatureTransformer()
    feature_transformer.remove_features(df,
                                        df_features,
                                        remove_null_features)
    main_pipe.add("Remove unresolvable null features",
                  feature_transformer)

    del feature_transformer

In [16]:
missing_table = missing_values_table(df)
display(missing_table)
nan_features = missing_table.index.to_list()
nan_features

Empty DataFrame
Columns: [Missing Values, % of Total Values]
Index: []

[]

# Data Cleaning

In [17]:
data_cleaner = FeatureDataCleaner()

In [18]:
data_cleaner.run_widget(df,
                        df_features,
                        nan_feature_names=df.columns.to_list())

interactive(children=(Select(description='Features', layout=Layout(height='175px', width='50%'), options=('Gen…

In [19]:
raise ValueError("Interact with widget before continuing")

ValueError: Interact with widget before continuing

In [ ]:
data_cleaner.perform_saved_widget_input(df,
                                        df_features)

In [ ]:
missing_table = missing_values_table(df)
display(missing_table)
remaing_nan_features = missing_table.index.to_list()
remaing_nan_features

In [ ]:
main_pipe.add("Cleaning features with methods that only apply to that one feature.",
              data_cleaner)

## Enforce correct data types

# String Cleaning

In [ ]:
df_features.display_features()

# Dummy encode data

In [ ]:
data_encoder = DataEncoder()

In [ ]:
data_encoder.apply_value_representation(df,
                                        df_features)

In [ ]:
qualitative_features = df_features.string_features() | df_features.categorical_features()

In [ ]:
data_encoder.make_dummies(df,
                          df_features,
                          qualitative_features=qualitative_features)

In [ ]:
df_features.display_features()

In [ ]:
data_encoder.make_values_bool(df,
                              df_features)

In [ ]:
df

In [ ]:
main_pipe.add("Ensure values are in proper form; convert proper values to dummies!",
              data_encoder)

# Test if the pipeline structure works on samples of the data and the entire set of the data

In [ ]:
main_tmp_df = pd.read_csv(dataset_path)

In [ ]:
sizes = [1,main_tmp_df.shape[0],5,15,20]

In [ ]:
for size in sizes:
    tmp_df_features = DataFrameTypes()
    tmp_df_features.init_on_json_file(os.getcwd() + f"/eflow Data/{dataset_name}/df_features.json")
    tmp_df = main_tmp_df[0:size]
    failure_found = False
    
    main_pipe.perform_pipeline(tmp_df,
                               tmp_df_features)
    
    print(f"size of test data: {size}")
    
    if df_features == tmp_df_features:
        print("df_features PASSED TEST! Properly changed in the pipeline!")
    else:
        print("df_features FAILED TEST! Didn't properly change in the pipeline")
        failure_found = True

    if set(df.columns) == set(tmp_df.columns):
        print("Field name check PASSED TEST! Field names were as expected.")
    else:
        print("Field name check FAILED TEST! Field names did differ from expected.")
        failure_found = True
        
    if failure_found:
        print("tmp_df")
        display(tmp_df)
        print()
        print("df")
        display(df[0:size])

    print("***" * 10)
    print()

In [ ]:
df_features.display_features()